# Projet Final Apache Spark

**Nom Etudiant : DIATTA**  

**Prenom Etudiant: HAMATH DIEUDONNE**  

**Classe : MASTER 1 BIG DATA**  


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public ou privé) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Jeudi 10 janvier 2021** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [2]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [45]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

head : Le terme head n'est pas reconnu comme nom d'applet de commande, fonction, fichier de script ou programme 


excutable. Vrifiez l'orthographe du nom, ou si un chemin d'accs existe, vrifiez que le chemin d'accs est correct 


et ressayez.


Au caractre Ligne:1 : 1


+ head -1 "datasets/sf-fire/sf-fire-calls.csv"


+ ~~~~


    + CategoryInfo          : ObjectNotFound: (head:String) [], CommandNotFoundException


    + FullyQualifiedErrorId : CommandNotFoundException


 




Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [3]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...

In [14]:
val sfFireFile = "sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [5]:
fireDF.cache()

res1: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [6]:
fireDF.count()

res2: Long = 175296


In [7]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [8]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [21]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [10]:
// Reponse 1

fireDF.na.drop.agg(countDistinct(col("CallType")).alias("count")).show()

+-----+
|count|
+-----+
|   28|
+-----+



### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [11]:
// Reponse 2
/*
Ecrire ici votre code
*/
val fewFireDF = fireDF
  .select("CallType")
fewFireDF.show()

+----------------+
|        CallType|
+----------------+
|  Structure Fire|
|Medical Incident|
|Medical Incident|
|    Vehicle Fire|
|          Alarms|
|  Structure Fire|
|          Alarms|
|          Alarms|
|Medical Incident|
|Medical Incident|
|Medical Incident|
|  Structure Fire|
|Medical Incident|
|Medical Incident|
|  Structure Fire|
|  Structure Fire|
|  Structure Fire|
|Medical Incident|
|Medical Incident|
|Medical Incident|
+----------------+
only showing top 20 rows



fewFireDF: org.apache.spark.sql.DataFrame = [CallType: string]


### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [11]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
 .select("IncidentNumber", "AvailableDtTm", "CallType", "ResponseDelayedinMins") 
  .where($"ResponseDelayedinMins" > 5)
newFireDF.show()

+--------------+--------------------+--------------------+---------------------+
|IncidentNumber|       AvailableDtTm|            CallType|ResponseDelayedinMins|
+--------------+--------------------+--------------------+---------------------+
|       2003409|01/11/2002 04:34:...|    Medical Incident|                 5.35|
|       2003642|01/12/2002 01:23:...|    Medical Incident|                 6.25|
|       2003818|01/13/2002 01:51:...|    Medical Incident|                  5.2|
|       2004152|01/14/2002 08:16:...|Citizen Assist / ...|                  5.6|
|       2004216|01/14/2002 12:27:...|    Medical Incident|                 7.25|
|       2004408|01/15/2002 06:53:...|    Medical Incident|            11.916667|
|       2004521|01/15/2002 03:18:...|    Medical Incident|             5.116667|
|       2004528|01/15/2002 03:55:...|    Medical Incident|             8.633333|
|       2004528|01/15/2002 07:49:...|    Medical Incident|             95.28333|
|       2004610|01/15/2002 0

newFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 2 more fields]


### Transformations des dates
Maintenant nous allons d'abord:
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard
2. Retourner le Dataframe transformée
3. Mettre en cache le nouveau DataFrame

In [35]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")
fireTSDF
.select("IncidentDate", "OnWatchDate", "AvailableDtTS")
.show()

fireTSDF.cache()

org.apache.spark.sql.AnalysisException:  cannot resolve '`CallDate`' given input columns: [IncidentNumber, AvailableDtTm, CallType, ResponseDelayedinMins];;

### Question 4
**Quels sont les types d'appels les plus courants?**

In [69]:
//Reponse 4
newFireDF
    .select("CallType")
    .where(col("CallType").isNotNull)
    .groupBy("CallType")
    .count()
    .orderBy(desc("count"))
    .show()

+--------------------+-----+
|            CallType|count|
+--------------------+-----+
|    Medical Incident|18981|
|      Structure Fire| 2424|
|              Alarms| 1797|
|   Traffic Collision| 1193|
|Citizen Assist / ...|  803|
|               Other|  585|
|        Water Rescue|  332|
|        Outside Fire|  305|
|Gas Leak (Natural...|  167|
|Odor (Strange / U...|  164|
|        Vehicle Fire|  123|
|   Electrical Hazard|  121|
|Elevator / Escala...|   94|
|Smoke Investigati...|   94|
|              HazMat|   76|
|          Fuel Spill|   64|
|Industrial Accidents|   33|
|Train / Rail Inci...|   32|
|Watercraft in Dis...|   21|
|   High Angle Rescue|   16|
+--------------------+-----+
only showing top 20 rows



### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [18]:
//Reponse 5-a
/*
Ecrire ici votre code
*/
fireDF
    .select("CallType")
    .where(col("CallType").isNotNull)
    .groupBy("CallType")
    .count()
    .orderBy(desc("count"))
    .show()

+--------------------+------+
|            CallType| count|
+--------------------+------+
|    Medical Incident|113794|
|      Structure Fire| 23319|
|              Alarms| 19406|
|   Traffic Collision|  7013|
|Citizen Assist / ...|  2524|
|               Other|  2166|
|        Outside Fire|  2094|
|        Vehicle Fire|   854|
|Gas Leak (Natural...|   764|
|        Water Rescue|   755|
|Odor (Strange / U...|   490|
|   Electrical Hazard|   482|
|Elevator / Escala...|   453|
|Smoke Investigati...|   391|
|          Fuel Spill|   193|
|              HazMat|   124|
|Industrial Accidents|    94|
|           Explosion|    89|
|Train / Rail Inci...|    57|
|  Aircraft Emergency|    36|
+--------------------+------+
only showing top 20 rows



### Question 5-b
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [1]:
//Reponse 5-b
/*
Ecrire ici votre code
*/
fireDF
    .select(col("Address"),
     col("Zipcode"))
    .where(col("Zipcode") == 94102)
    .show()

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-Q764EIG:4040
SparkContext available as 'sc' (version = 2.4.7, master = local[*], app id = local-1610226484268)
SparkSession available as 'spark'


<console>: 30: error: not found: value fireDF

### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [24]:
//Reponse 6
/*
Ecrire ici votre code
*/
newFireDF.describe().show(false)

+-------+-----------------+----------------------+----------------------+---------------------+
|summary|IncidentNumber   |AvailableDtTm         |CallType              |ResponseDelayedinMins|
+-------+-----------------+----------------------+----------------------+---------------------+
|count  |27508            |27189                 |27508                 |27508                |
|mean   |9918547.19547041 |null                  |null                  |9.677239351968042    |
|stddev |5573063.988420601|null                  |null                  |22.68852710610722    |
|min    |30707            |01/01/2001 01:45:54 AM|Administrative        |5.016667             |
|max    |18130265         |12/31/2017 12:55:42 PM|Watercraft in Distress|1844.55              |
+-------+-----------------+----------------------+----------------------+---------------------+



### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [18]:
//Reponse 7-a
/*
Ecrire ici votre code
*/

### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [19]:
//Reponse 7-b
/*
Ecrire ici votre code
*/

### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [24]:
//Reponse 8
newFireDF
    .select("Address")
    .where(col("ResponseDelayedinMins").isNotNull)
    .orderBy(desc("ResponseDelayedinMins"))
    .show()

<console>: 36: error: not found: value newFireDF

### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [25]:
//Reponse 9
/*
Ecrire ici votre code
*/
newFireDF.write.parquet("tableau.parquet")

org.apache.spark.SparkException:  Job aborted.

### Question 10
**Comment relire les données stockée en format Parquet?**

In [11]:
//Reponse 10

val parquetFileDF = spark.read.parquet("tableau.parquet")


org.apache.spark.sql.AnalysisException:  Unable to infer schema for Parquet. It must be specified manually.;

## FIN